# Project: Using Pandas to Solve Business Problems

The purpose of this project is to demonstrate the use of the Pandas library to help solve business problems.

## Importing Packages

In [1]:
import pandas as pd

## Importing Datasets

For this project we will use the Legos datasets from DataCamp, which consist of data on every LEGO set that has ever been sold; the names of the sets, what bricks they contain, what color the bricks are, etc.

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/KeithGalli/lego-analysis/master/datasets/lego_sets.csv')
parent_theme = pd.read_csv('https://raw.githubusercontent.com/KeithGalli/lego-analysis/master/datasets/parent_themes.csv')

In [3]:
df.head()

,set_num,name,year,num_parts,theme_name,parent_theme
0,00-1,Weetabix Castle,1970,471.0,Castle,Legoland
1,0011-2,Town Mini-Figures,1978,NaN,Supplemental,Town
2,0011-3,Castle 2 for 1 Bonus Offer,1987,NaN,Lion Knights,Castle
3,0012-1,Space Mini-Figures,1979,12.0,Supplemental,Space
4,0013-1,Space Mini-Figures,1979,12.0,Supplemental,Space


In [4]:
parent_theme.head()

,id,name,is_licensed
0,1,Technic,False
1,22,Creator,False
2,50,Town,False
3,112,Racers,False
4,126,Space,False


## Merging Datasets

In [27]:
lego_df = pd.merge(left = df, right = parent_theme, how = 'inner', left_on = 'parent_theme', right_on = 'name')

## Data Cleaning

In [28]:
# Dropping duplicate column
lego_df.drop('name_y', axis = 1, inplace = True)

# Renaming column
lego_df.rename(columns = {'name_x': 'name'}, inplace = True)
lego_df.head()

,set_num,name,year,num_parts,theme_name,parent_theme,id,is_licensed
0,00-1,Weetabix Castle,1970,471.0,Castle,Legoland,411,False
1,00-2,Weetabix Promotional House 1,1976,NaN,Building,Legoland,411,False
2,00-3,Weetabix Promotional House 2,1976,NaN,Building,Legoland,411,False
3,00-4,Weetabix Promotional Windmill,1976,126.0,Building,Legoland,411,False
4,00-7,Weetabix Promotional Lego Village,1976,NaN,Building,Legoland,411,False


In [29]:
# Dropping rows that have NaNs for critical fields
lego_df.dropna(subset = ['set_num'], inplace = True)

In [30]:
lego_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11833 entries, 0 to 11985
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   set_num       11833 non-null  object 
 1   name          11833 non-null  object 
 2   year          11833 non-null  int64  
 3   num_parts     6835 non-null   float64
 4   theme_name    11833 non-null  object 
 5   parent_theme  11833 non-null  object 
 6   id            11833 non-null  int64  
 7   is_licensed   11833 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(4)
memory usage: 751.1+ KB


## 1) What percentage of all licensed Lego sets ever released were Star Wars themed?

In [31]:
licensed_sets = lego_df[lego_df['is_licensed'] == True]
licensed_sets.head()

,set_num,name,year,num_parts,theme_name,parent_theme,id,is_licensed
3493,10018-1,Darth Maul,2001,1868.0,Star Wars,Star Wars,158,True
3494,10019-1,Rebel Blockade Runner - UCS,2001,NaN,Star Wars Episode 4/5/6,Star Wars,158,True
3495,10026-1,Naboo Starfighter - UCS,2002,NaN,Star Wars Episode 1,Star Wars,158,True
3496,10030-1,Imperial Star Destroyer - UCS,2002,3115.0,Star Wars Episode 4/5/6,Star Wars,158,True
3497,10123-1,Cloud City,2003,707.0,Star Wars Episode 4/5/6,Star Wars,158,True


In [32]:
num_starwars_sets = len(licensed_sets[licensed_sets['parent_theme'] == 'Star Wars'])
num_total_sets = len(licensed_sets)

percent_starwars = (num_starwars_sets / num_total_sets) * 100

print('{}% of all licensed Lego sets ever released were Stars Wars themed.'.format(round(percent_starwars)))

52% of all licensed Lego sets ever released were Stars Wars themed.


## 2) In which year was Star Wars not the most popular licensed theme (in terms of number of sets released that year)?

In [44]:
# Grouping data to return numer of released licensed theme Lego sets per year
group_data = licensed_sets[['year', 'set_num', 'parent_theme']].groupby(by = ['year', 'parent_theme']).count().reset_index()
group_data

,year,parent_theme,set_num
0,1999,Star Wars,13
1,2000,Disney's Mickey Mouse,5
2,2000,Star Wars,26
3,2001,Harry Potter,11
4,2001,Star Wars,14
...,...,...,...
78,2017,Disney Princess,6
79,2017,Minecraft,9
80,2017,Pirates of the Caribbean,1
81,2017,Star Wars,55


In [52]:
# Selecting most popular licensed theme per year
group_data = group_data.sort_values('set_num', ascending = False).drop_duplicates('year')
group_data.sort_values('year', inplace = True)
group_data

,year,parent_theme,set_num
0,1999,Star Wars,13
2,2000,Star Wars,26
4,2001,Star Wars,14
6,2002,Star Wars,28
9,2003,Star Wars,32
12,2004,Star Wars,20
16,2005,Star Wars,28
20,2006,Star Wars,11
24,2007,Star Wars,16
28,2008,Star Wars,23


In [68]:
non_starwars_year = group_data[group_data['parent_theme'] != 'Star Wars']['year'].values[0]
non_starwars_year

2017

In [69]:
print('Star Wars was not the most popular licensed Lego theme in {}.'.format(non_starwars_year))

Star Wars was not the most popular licensed Lego theme in 2017.
